In [ ]:
from src.midi.midiToNotes import MidiProcessor
from src.piano.piano import Piano
from src.hand.hand import Hand
from src.hand.finger import Finger
from src.recorder.recorder import Recorder
from src.recorder.recorderPool import RecorderPool

midi_name = "World is Mine - Hatsune Miku"
track_numbers = [0]
channel_number = 0
hand_range = 13
finger_range = 4
FPS = 60

# 初始各种文件路径
hand_recorder_file_path = f"output/hand_recorders/{midi_name}.hand"

# 读取midi文件，获得音高和时间轴
midi_processor = MidiProcessor(midi_name, track_numbers, channel_number,FPS)
notes_maps = midi_processor.generate_notes_map_and_messages()

# 初始化钢琴
piano = Piano()

# 初始化左右手,左手放在C3，右手放在C4
left_hand = Hand([Finger(0, piano.note_to_key(48)), Finger(1, piano.note_to_key(50)), Finger(2, piano.note_to_key(52)), Finger(3, piano.note_to_key(53)), Finger(4, piano.note_to_key(55))], piano, True)
right_hand = Hand([Finger(0, piano.note_to_key(60), False), Finger(1, piano.note_to_key(62), False), Finger(
    2, piano.note_to_key(64), False), Finger(3, piano.note_to_key(65), False), Finger(4, piano.note_to_key(67), False)], piano, False)

init_real_tick = 0.0
init_real_ticks = []

# 初始化recorder
recorder = Recorder(piano, [left_hand], [right_hand],
                    0, init_real_tick, init_real_ticks)

# 初始化recorderPool
pool_size = 100
recorder_pool = RecorderPool([recorder],pool_size,0)

notes_map 保存到了 asset/temp/notes_map.json
pitch_wheel_map 保存到了 asset/temp/pitch_wheel_map.json
messages 保存到了 asset/temp/messages.json
全曲的速度变化是:
在0轨，tick为0时，速度变为375000

全曲的每拍tick数是:384

如果以60的fps做成动画，一共是259012 ticks, 合计15176.484375帧, 约252.94140625秒


In [ ]:
# 测试用模块，正常运行时跳过此段代码
from src.midi.midiToNotes import NotesMap

current_real_tick = 167616
current_notes = [63, 66, 70, 73, 77]
current_notes_map = NotesMap(
    notes=current_notes, real_tick=current_real_tick)

recorder_pool.update_recorder_pool(current_notes_map, hand_range, finger_range)

In [2]:
from tqdm import tqdm

# 使用notes_map，开始迭代更新recorder，并且放入recorder_pool中
for notes_map in tqdm(notes_maps, desc="生成指法中……", unit="step"):
    recorder_pool.update_recorder_pool(notes_map, hand_range, finger_range)

recorder_pool.export_pool_info(hand_recorder_file_path)

处理notes_maps: 100%|██████████| 821/821 [03:23<00:00,  4.02map/s]

最优记录的熵值为：12090
已保存至output/hand_recorders/World is Mine - Hatsune Miku.hand
